In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import pickle
import numpy as np
import os

knowledge_points_path = os.path.join(os.getcwd(), "knowledge_points.pkl")
with open(knowledge_points_path, 'rb') as f_knowledge_points:
    knowledge_points = pickle.load(f_knowledge_points)
    
knowledge_points

['“重农抑商”政策',
 '不完全显性',
 '与细胞分裂有关的细胞器',
 '中央官制——三公九卿制',
 '中心体的结构和功能',
 '人体免疫系统在维持稳态中的作用',
 '人体水盐平衡调节',
 '人体的体温调节',
 '人口增长与人口问题',
 '人口迁移与人口流动',
 '人工授精、试管婴儿等生殖技术',
 '伴性遗传',
 '体液免疫的概念和过程',
 '免疫系统的功能',
 '免疫系统的组成',
 '兴奋在神经元之间的传递',
 '兴奋在神经纤维上的传导',
 '内环境的稳态',
 '内质网的结构和功能',
 '农业区位因素',
 '减数分裂与有丝分裂的比较',
 '减数分裂的概念',
 '劳动就业与守法经营',
 '器官移植',
 '地球所处的宇宙环境',
 '地球的内部圈层结构及特点',
 '地球的外部圈层结构及特点',
 '地球运动的地理意义',
 '地球运动的基本形式',
 '垄断组织的出现',
 '培养基与无菌技术',
 '基因工程的原理及技术',
 '基因工程的概念',
 '基因的分离规律的实质及应用',
 '基因的自由组合规律的实质及应用',
 '复等位基因',
 '夏商两代的政治制度',
 '太阳对地球的影响',
 '工业区位因素',
 '拉马克的进化学说',
 '文艺的春天',
 '核糖体的结构和功能',
 '海峡两岸关系的发展',
 '液泡的结构和功能',
 '清末民主革命风潮',
 '溶酶体的结构和功能',
 '激素调节',
 '生命活动离不开细胞',
 '生态系统的营养结构',
 '生物工程技术',
 '生物性污染',
 '生物技术在其他方面的应用',
 '皇帝制度',
 '社会主义市场经济的伦理要求',
 '社会主义是中国人民的历史性选择',
 '神经调节和体液调节的比较',
 '第三产业的兴起和“新经济”的出现',
 '组成细胞的化合物',
 '组成细胞的化学元素',
 '细胞大小与物质运输的关系',
 '细胞有丝分裂不同时期的特点',
 '细胞的多样性和统一性',
 '群落的结构',
 '胚胎移植',
 '蛋白质的合成',
 '血糖平衡的调节',
 '走进细胞',
 '选官、用官制度的变化',
 '遗传的分子基础',
 '遗传的细胞基础',
 '避孕的原理和方法',
 '郡县制',
 '高尔基体的结构和功能']

In [2]:
words_path = os.path.join(os.getcwd(), "words.pkl")
with open(words_path, 'rb') as f_words:
    words = pickle.load(f_words)
words

In [3]:
'''
    1.输入是序列中token的embedding与位置embedding
    2.token的embedding与其位置embedding相加，得到一个vector(这个向量融合了token与position信息)
    3.在2之前，token的embedding乘上一个scale(防止点积变大，造成梯度过小)向量[sqrt(emb_dim)]，这个假设为了减少embedding中的变化，没有这个scale，很难稳定的去训练model。
    4.加入dropout
    5.通过N个encoder layer，得到Z。此输出Z被传入一个全连接层作分类。
    src_mask对于非<pad>值为1,<pad>为0。为了计算attention而遮挡<pad>这个无意义的token。与source 句子shape一致。
'''
class TransformerEncoder(nn.Module):
    def __init__(self, input_dim, output_dim, emb_dim, n_layers, n_heads, pf_dim, dropout, position_length, pad_idx):
        super(TransformerEncoder, self).__init__()
        
        self.pad_idx = pad_idx
        self.scale = torch.sqrt(torch.FloatTensor([emb_dim]))

        # 词的embedding
        self.token_embedding = nn.Embedding(input_dim, emb_dim)
        # 对词的位置进行embedding
        self.position_embedding = nn.Embedding(position_length, emb_dim)
        # encoder层，有几个encoder层，每个encoder有几个head
        self.layers = nn.ModuleList([EncoderLayer(emb_dim, n_heads, pf_dim, dropout) for _ in range(n_layers)])
        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(emb_dim, output_dim)
        self.sigmoid = nn.Sigmoid()

    def mask_src_mask(self, src):
        # src=[batch_size, src_len]

        # src_mask=[batch_size, 1, 1, src_len]
        src_mask = (src != self.pad_idx).unsqueeze(1).unsqueeze(2)
        return src_mask
    
    def forward(self, src):
        # src=[batch_size, seq_len]
        # src_mask=[batch_size, 1, 1, seq_len]
        src_mask = self.mask_src_mask(src)
        
        batch_size = src.shape[0]
        src_len = src.shape[1]

        # 构建位置tensor -> [batch_size, seq_len]，位置序号从(0)开始到(src_len-1)
        position = torch.arange(0, src_len).unsqueeze(0).repeat(batch_size, 1)

        # 对词和其位置进行embedding -> [batch_size,seq_len,embdim]
        token_embeded = self.token_embedding(src) * self.scale
        position_embeded = self.position_embedding(position)

        # 对词和其位置的embedding进行按元素加和 -> [batch_size, seq_len, embdim]
        src = self.dropout(token_embeded + position_embeded)

        for layer in self.layers:
            src = layer(src, src_mask)

        # [batch_size, seq_len, emb_dim] -> [batch_size, output_dim]
        src = src.permute(0, 2, 1)
        src = torch.sum(src, dim=-1)
        src = self.fc(src)
        src = self.sigmoid(src)
        return src

'''
encoder layers：
    1.将src与src_mask传入多头attention层(multi-head attention)
    2.dropout
    3.使用残差连接后传入layer-norm层(输入+输出后送入norm)后得到的输出
    4.输出通过前馈网络feedforward层
    5.dropout
    6.一个残差连接后传入layer-norm层后得到的输出喂给下一层
    注意：
        layer之间不共享参数
        多头注意力层用到的是多个自注意力层self-attention
'''
class EncoderLayer(nn.Module):
    def __init__(self, emb_dim, n_heads, pf_dim, dropout):
        super(EncoderLayer, self).__init__()
        # 注意力层后的layernorm
        self.self_attn_layer_norm = nn.LayerNorm(emb_dim)
        # 前馈网络层后的layernorm
        self.ff_layer_norm = nn.LayerNorm(emb_dim)
        # 多头注意力层
        self.self_attention = MultiHeadAttentionLayer(emb_dim, n_heads, dropout)
        # 前馈层
        self.feedforward = FeedforwardLayer(emb_dim, pf_dim, dropout)

        self.dropout = nn.Dropout(dropout)

    def forward(self, src, src_mask):
        #src=[batch_size, seq_len, emb_dim]
        #src_mask=[batch_size, 1, 1, seq_len]

        # self-attention
        # _src=[batch size, query_len, emb_dim]
        _src, _ = self.self_attention(src, src, src, src_mask)

        # dropout, 残差连接以及layer-norm
        # src=[batch_size, seq_len, emb_dim]
        src = self.self_attn_layer_norm(src + self.dropout(_src))

        # 前馈网络
        # _src=[batch_size, seq_len, emb_dim]
        _src = self.feedforward(src)

        # dropout, 残差连接以及layer-norm
        # src=[batch_size, seq_len, emb_dim]
        src = self.ff_layer_norm(src + self.dropout(_src))

        return src
'''
多头注意力层的计算:
    1.q,k,v的计算是通过线性层fc_q,fc_k,fc_v
    2.对query,key,value的emb_dim split成n_heads
    3.通过计算Q*K/scale计算energy
    4.利用mask遮掩不需要关注的token
    5.利用softmax与dropout
    6.5的结果与V矩阵相乘
    7.最后通过一个前馈fc_o输出结果
注意:Q,K,V的长度一致
'''
class MultiHeadAttentionLayer(nn.Module):
    def __init__(self, emb_dim, n_heads, dropout):
        super(MultiHeadAttentionLayer, self).__init__()
        assert emb_dim % n_heads == 0
        self.emb_dim = emb_dim
        self.n_heads = n_heads
        self.head_dim = emb_dim//n_heads

        self.fc_q = nn.Linear(emb_dim, emb_dim)
        self.fc_k = nn.Linear(emb_dim, emb_dim)
        self.fc_v = nn.Linear(emb_dim, emb_dim)

        self.fc_o = nn.Linear(emb_dim, emb_dim)

        self.dropout = nn.Dropout(dropout)

        self.scale = torch.sqrt(torch.FloatTensor([self.head_dim]))

    def forward(self, query, key, value, mask=None):
        # query=[batch_size, query_len, emb_dim]
        # key=[batch_size, key_len, emb_dim]
        # value=[batch_size, value_len, emb_dim]
        batch_size = query.shape[0]

        # Q=[batch_size, query_len, emb_dim]
        # K=[batch_size, key_len, emb_dim]
        # V=[batch_size, value_len, emb_dim]
        Q = self.fc_q(query)
        K = self.fc_k(key)
        V = self.fc_v(value)

        '''
        view与reshape的异同：
        
        torch的view()与reshape()方法都可以用来重塑tensor的shape，区别就是使用的条件不一样。view()方法只适用于满足连续性条件的tensor，并且该操作不会开辟新的内存空间，
        只是产生了对原存储空间的一个新别称和引用，返回值是视图。而reshape()方法的返回值既可以是视图，也可以是副本，当满足连续性条件时返回view，
        否则返回副本[ 此时等价于先调用contiguous()方法在使用view() ]。因此当不确能否使用view时，可以使用reshape。如果只是想简单地重塑一个tensor的shape，
        那么就是用reshape，但是如果需要考虑内存的开销而且要确保重塑后的tensor与之前的tensor共享存储空间，那就使用view()。
        '''

        # Q=[batch_size, n_heads, query_len, head_dim]
        # K=[batch_size, n_heads, key_len, head_dim]
        # V=[batch_size, n_heads, value_len, head_dim]
        Q = Q.view(batch_size, -1, self.n_heads, self.head_dim).permute(0, 2, 1, 3)
        K = K.view(batch_size, -1, self.n_heads, self.head_dim).permute(0, 2, 1, 3)
        V = V.view(batch_size, -1, self.n_heads, self.head_dim).permute(0, 2, 1, 3)

        # 注意力打分矩阵 [batch_size, n_heads, query_len, head_dim] * [batch_size, n_heads, head_dim, key_len] = [batch_size, n_heads, query_len, key_len]
        energy = torch.matmul(Q, K.permute(0, 1, 3, 2)) / self.scale

        if mask is not None:
            energy = energy.masked_fill(mask == 0, -1e10)
        # [batch_size, n_heads, query_len, key_len]
        attention = torch.softmax(energy , dim = -1)

        # [batch_size, n_heads, query_len, key_len]*[batch_size, n_heads, value_len, head_dim]=[batch_size, n_heads, query_len, head_dim]
        x = torch.matmul(self.dropout(attention), V)

        # [batch_size, query_len, n_heads, head_dim]
        x = x.permute(0, 2, 1, 3).contiguous()

        # [batch_size, query_len, emb_dim]
        x = x.view(batch_size, -1, self.emb_dim)

        # [batch_size, query_len, emb_dim]
        x = self.fc_o(x)

        return x, attention

'''
前馈层
'''
class FeedforwardLayer(nn.Module):
    def __init__(self, emb_dim, pf_dim, dropout):
        super(FeedforwardLayer, self).__init__()
        self.fc_1 = nn.Linear(emb_dim, pf_dim)
        self.fc_2 = nn.Linear(pf_dim, emb_dim)

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # x=[batch_size, seq_len, emb_dim]

        # x=[batch_size, seq_len, pf_dim]
        x = self.dropout(torch.relu(self.fc_1(x)))

        # x=[batch_size, seq_len, emb_dim]
        x = self.fc_2(x)

        return x

In [4]:

input_dim = len(words) 
output_dim = 73 # 共73个知识点标签
emb_dim = 256
n_layers = 3
n_heads = 8
pf_dim = 512
dropout = 0.1
position_length = 200

# <pad>
pad_index = words['<pad>']

# 构建model
model = TransformerEncoder(input_dim, output_dim, emb_dim, n_layers, n_heads, pf_dim, dropout, position_length, pad_index)
model_path = os.path.join(os.getcwd(), "model.h5")
model.load_state_dict(torch.load(model_path))

#checkpoint = torch.load(model_path)
#model.load_state_dict({k.replace('module.',''):v for k,v in checkpoint.items()})

<All keys matched successfully>

In [6]:
# 加载停词
parent_path = os.path.abspath(os.path.join(os.getcwd(), '..'))
stopwords_path = os.path.join(parent_path,  'data', 'stopwords.txt')

stopwords_set = set()
with open(stopwords_path, 'r', encoding='utf-8') as f_read:
    for line in f_read:
        stopwords_set.add(line.strip())

print('stop words len :{}'.format(len(stopwords_set)))

stop words len :859


In [7]:
import jieba
import re

def content_preprocess(content):
    # 去标点
    r = re.compile("[^\u4e00-\u9fa5]+|题目")
    content = r.sub("", content)  # 删除所有非汉字字符
    # jieba分词
    words = jieba.cut(content, cut_all=False)
    words = [w for w in words if w not in stopwords_set]
    # words = ' '.join(words)
    return words

segment = content_preprocess('下表是美国、欧共体和日本的国民经济占世界经济总量的比例表（单位：亿美元）。结合所学知识判断，对此分析不正确的是（   ）年份美国欧共体日本195634.88%15.85%4.26%197327.08%27.017%18.023%A. 福利政策的实施已然严重阻碍了美国经济发展B. 美、欧、日三足鼎立之势有力冲击着两极格局C. 越南战争已将美国经济拖入了‘滞胀”的轨道D. 欧、日经济的迅速发展威胁着美国的霸主地位题型')


Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\User\AppData\Local\Temp\jieba.cache
Loading model cost 0.603 seconds.
Prefix dict has been built successfully.


In [8]:


def bow(sentence):
    sentence_words = content_preprocess(sentence)
    indexed = [words.stoi[t] for t in sentence_words]
    src_tensor = torch.LongTensor(indexed)
    src_tensor = src_tensor.unsqueeze(0)
    return src_tensor

def predict_class(sentence):
    sentence_bag = bow(sentence)
    model.eval()
    with torch.no_grad():
        outputs = model(sentence_bag)
    print('outputs:{}'.format(outputs))
    predicts = outputs.data.numpy() > 0.15
    predicts = predicts.astype(int)
    print('predict result:{}'.format(predicts))
  
    return predicts

def predict(text):
    predict_result = predict_class(text)
    return predict_result

result = predict('秦始皇统一六国后创制了一套御玺。如任命国家官员，则封印“皇帝之玺”；若任命四夷的官员，则用“天子之玺”；信玺用于对国内和四夷用兵事宜，行玺则为皇帝外巡时随身携带。材料不能说明（   ）A. 皇帝处于至高无上的地位B. 秦朝有内外两种系统处理国事C. 秦朝实行中央集权的体制D. 三公九卿制大大提升行政效率题型: 单选题|难度: 一般|使用次数: 0|纠错复制收藏到空间加入选题篮查看答案解析答案：D解析：本题要求选择否定项，据材料提到，秦始皇统一六国后创制了一套御玺，如任命国家官员，则封印“皇帝之玺”……，结合所学知识可知，这说明皇帝处于至高无上的地位，故A正确，排除。信玺和行玺的区别说明秦朝有内外两种系统处理国事，故B正确，排除。材料也说明秦朝实行中央集权的体制，故C正确，排除。材料未涉及三公九卿制大大提升行政效率，故D错误，符合题意')


outputs:tensor([[1.9926e-01, 3.5290e-02, 3.9332e-03, 1.9053e-01, 3.5161e-03, 1.6357e-02,
         4.1133e-02, 4.7413e-02, 3.7640e-02, 3.0976e-02, 2.5746e-02, 1.4051e-03,
         1.0409e-02, 7.8402e-03, 1.3440e-02, 3.5521e-02, 3.0062e-02, 2.3890e-02,
         2.3336e-03, 8.1141e-02, 3.0676e-04, 1.5338e-04, 1.9953e-01, 6.5439e-02,
         2.3860e-03, 7.0703e-03, 8.9453e-03, 6.5006e-02, 6.0722e-03, 1.1673e-01,
         7.6648e-02, 1.0720e-01, 1.9201e-01, 9.0504e-03, 2.5807e-02, 1.8517e-01,
         1.9134e-01, 2.9159e-02, 1.2422e-01, 9.5979e-02, 1.6358e-01, 1.8997e-03,
         1.2531e-01, 1.2103e-03, 9.5366e-02, 2.6855e-03, 1.0009e-01, 5.8413e-03,
         1.0218e-01, 1.3180e-01, 1.2218e-02, 2.3062e-01, 1.6375e-01, 3.0377e-01,
         2.1144e-01, 2.4156e-02, 1.9811e-01, 4.7713e-02, 5.0506e-02, 6.0502e-02,
         9.3408e-04, 2.6121e-03, 9.4636e-02, 7.4718e-02, 3.3602e-02, 3.3723e-02,
         4.3755e-03, 6.3906e-02, 2.7815e-02, 3.1955e-02, 5.1422e-03, 1.9598e-01,
         2.4030e-03]

In [10]:
res = result.flatten()
predict_index = np.argwhere(res==1)
predict_index=predict_index.flatten()
knowledge_points = np.array(knowledge_points)
predict_tag = knowledge_points[predict_index]
' '.join(predict_tag.tolist())

'“重农抑商”政策 中央官制——三公九卿制 劳动就业与守法经营 基因工程的概念 复等位基因 夏商两代的政治制度 文艺的春天 生物技术在其他方面的应用 皇帝制度 社会主义市场经济的伦理要求 社会主义是中国人民的历史性选择 第三产业的兴起和“新经济”的出现 郡县制'